<a href="https://colab.research.google.com/github/raviteja41/EVA4/blob/master/Eva4_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install torchsummary
from torchsummary import summary
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3)   #input channel size-28 Output channel size- 26 
        self.conv2 = nn.Conv2d(8, 16, 3)  #input channel size-26 Output channel size- 24 
        self.conv3 = nn.Conv2d(16, 32, 3) #input channel size-24 Output channel size- 22 
        self.conv1_1 = nn.Conv2d(32, 8, 1)#input channel size-22 Output channel size- 22 
        self.pool1 = nn.MaxPool2d(2, 2)   #input channel size-22 Output channel size- 11 
        self.conv4 = nn.Conv2d(8, 16, 3)  #input channel size-11 Output channel size- 9  
        self.conv5 = nn.Conv2d(16, 32, 3) #input channel size-9  Output channel size- 7  
        self.conv6 = nn.Conv2d(32, 16, 3) #input channel size-7  Output channel size- 5  
        self.conv7 = nn.Conv2d(16, 10, 3) #input channel size-5  Output channel size- 3
        self.avg = nn.AdaptiveAvgPool2d(1)#input channel size-3  Output channel size- 1  
        self.dropout = nn.Dropout(p=0.09) 
        self.bn_1 = nn.BatchNorm2d(16)    
        self.bn_2 = nn.BatchNorm2d(32)    
        
    def forward(self, x):
        x = self.pool1(self.conv1_1(self.dropout(F.relu(self.bn_2(self.conv3(self.dropout(F.relu(self.bn_1(self.conv2(self.dropout(F.relu(self.conv1(x)))))))))))))
        x = self.conv7(self.dropout(F.relu(self.bn_1(self.conv6(self.dropout(F.relu(self.conv5(self.dropout(F.relu(self.conv4(x)))))))))))
        x = self.avg(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
           Dropout-2            [-1, 8, 26, 26]               0
            Conv2d-3           [-1, 16, 24, 24]           1,168
       BatchNorm2d-4           [-1, 16, 24, 24]              32
           Dropout-5           [-1, 16, 24, 24]               0
            Conv2d-6           [-1, 32, 22, 22]           4,640
       BatchNorm2d-7           [-1, 32, 22, 22]              64
           Dropout-8           [-1, 32, 22, 22]               0
            Conv2d-9            [-1, 8, 22, 22]             264
        MaxPool2d-10            [-1, 8, 11, 11]               0
           Conv2d-11             [-1, 16, 9, 9]           1,168
          Dropout-12             [-1, 16, 9, 9]               0
           Conv2d-13             [-1, 32, 7, 7]           4,640
          Dropout-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(10)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.9)

for epoch in range(20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0874, Accuracy: 9726/10000 (97%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0628, Accuracy: 9804/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0229, Accuracy: 9925/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0224, Accuracy: 9929/10000 (99%)



loss=0.0016700128326192498 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.03it/s]



Test set: Average loss: 0.0196, Accuracy: 9940/10000 (99%)

